In [1]:
# !pip3.10 install --pre torch==2.5.0.dev20240912+cu121 --index-url https://download.pytorch.org/whl/nightly/cu121

In [2]:
# !pip3.10 install torch -U

In [3]:
# !pip3.10 uninstall transformers -y

In [4]:
# !pip3.10 install -e . --no-deps

In [5]:
import json
import torch
import torch.nn.functional as F
import math
from glob import glob

In [6]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-small-malaysian-cased', local_files_only = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model = T5ForConditionalGeneration.from_pretrained('mesolitica/nanot5-small-malaysian-translation-v2',
                                                  attn_implementation = 'sdpa', local_files_only = True).to(torch.bfloat16)

In [8]:
encoder_emb = torch.nn.Embedding(
    model.config.relative_attention_num_buckets, 
    model.config.num_heads
)
encoder_emb.load_state_dict(
    model.encoder.block[0].layer[0].SelfAttention.relative_attention_bias.state_dict()
)

decoder_emb = torch.nn.Embedding(
    model.config.relative_attention_num_buckets, 
    model.config.num_heads
)
decoder_emb.load_state_dict(
    model.decoder.block[0].layer[0].SelfAttention.relative_attention_bias.state_dict()
)

<All keys matched successfully>

In [9]:
files = glob('/home/husein/ssd3/translation/train.json*.splitted')
len(files)

68

In [10]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import streaming
import numpy as np
from tqdm import tqdm
from glob import glob
import os
import json

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'data': 'str',
}
hashes = 'sha1', 'xxh64'

In [12]:
def _relative_position_bucket(relative_position, bidirectional=True, num_buckets=32, max_distance=128):
    """
    Adapted from Mesh Tensorflow:
    https://github.com/tensorflow/mesh/blob/0cb87fe07da627bf0b7e60475d59f95ed6b5be3d/mesh_tensorflow/transformer/transformer_layers.py#L593

    Translate relative position to a bucket number for relative attention. The relative position is defined as
    memory_position - query_position, i.e. the distance in tokens from the attending position to the attended-to
    position. If bidirectional=False, then positive relative positions are invalid. We use smaller buckets for
    small absolute relative_position and larger buckets for larger absolute relative_positions. All relative
    positions >=max_distance map to the same bucket. All relative positions <=-max_distance map to the same bucket.
    This should allow for more graceful generalization to longer sequences than the model has been trained on

    Args:
        relative_position: an int32 Tensor
        bidirectional: a boolean - whether the attention is bidirectional
        num_buckets: an integer
        max_distance: an integer

    Returns:
        a Tensor with the same shape as relative_position, containing int32 values in the range [0, num_buckets)
    """
    relative_buckets = 0
    if bidirectional:
        num_buckets //= 2
        relative_buckets += (relative_position > 0).to(torch.long) * num_buckets
        relative_position = torch.abs(relative_position)
    else:
        relative_position = -torch.min(relative_position, torch.zeros_like(relative_position))
    # now relative_position is in the range [0, inf)

    # half of the buckets are for exact increments in positions
    max_exact = num_buckets // 2
    is_small = relative_position < max_exact

    # The other half of the buckets are for logarithmically bigger bins in positions up to max_distance
    relative_position_if_large = max_exact + (
        torch.log(relative_position.float() / max_exact)
        / math.log(max_distance / max_exact)
        * (num_buckets - max_exact)
    ).to(torch.long)
    relative_position_if_large = torch.min(
        relative_position_if_large, torch.full_like(relative_position_if_large, num_buckets - 1)
    )

    relative_buckets += torch.where(is_small, relative_position, relative_position_if_large)
    return relative_buckets

def compute_bias(
    query_length, 
    key_length,
    relative_attention_bias,
    bidirectional = True, 
    num_buckets = 32, 
    max_distance = 128, 
    device=None,
):
    """Compute binned relative position bias"""
    if device is None:
        device = relative_attention_bias.weight.device
    context_position = torch.arange(query_length, dtype=torch.long, device=device)[:, None]
    memory_position = torch.arange(key_length, dtype=torch.long, device=device)[None, :]
    relative_position = memory_position - context_position  # shape (query_length, key_length)
    relative_position_bucket = _relative_position_bucket(
        relative_position,  # shape (query_length, key_length)
        bidirectional=bidirectional,
        num_buckets=num_buckets,
        max_distance=max_distance,
    )
    values = relative_attention_bias(relative_position_bucket)  # shape (query_length, key_length, num_heads)
    values = values.permute([2, 0, 1]).unsqueeze(0)  # shape (1, num_heads, query_length, key_length)
    return values

def block_diagonal_concat_inverted(*masks, dtype=torch.bfloat16):
    total_size = sum(mask.size(0) for mask in masks)
    combined_mask = torch.zeros(total_size, total_size, dtype=dtype)

    current_pos = 0

    for mask in masks:
        size = mask.size(0)
        combined_mask[current_pos:current_pos + size, current_pos:current_pos + size] = mask
        current_pos += size

    min_value = torch.finfo(dtype).min if dtype.is_floating_point else torch.iinfo(dtype).min
    inverted_mask = torch.where(combined_mask == 1, torch.tensor(0, dtype=dtype), min_value)
    return inverted_mask.unsqueeze(0)

def pad_attention_mask(attention_mask):
    maxlen_right = max([attention_mask[i].shape[1] for i in range(len(attention_mask))])
    maxlen_bottom = max([attention_mask[i].shape[0] for i in range(len(attention_mask))])
    attention_mask = [
        F.pad(
            attention_mask[i],
            (0, maxlen_right - attention_mask[i].shape[1], 0, maxlen_bottom - attention_mask[i].shape[0])) for i in range(
            len(attention_mask))]
    return torch.stack(attention_mask)

def pad_attention_mask_4d(attention_mask):
    maxlen_right = max([attention_mask[i].shape[-2] for i in range(len(attention_mask))])
    maxlen_bottom = max([attention_mask[i].shape[-1] for i in range(len(attention_mask))])
    attention_mask = [
        F.pad(
            attention_mask[i],
            (0, maxlen_right - attention_mask[i].shape[-2], 0, maxlen_bottom - attention_mask[i].shape[-1])) for i in range(
            len(attention_mask))]
    return torch.stack(attention_mask)

def block_diagonal_concat(*masks, dtype=torch.bfloat16):
    total_size = sum(mask.size(0) for mask in masks)
    combined_mask = torch.zeros(total_size, total_size, dtype=dtype)

    current_pos = 0

    for mask in masks:
        size = mask.size(0)
        combined_mask[current_pos:current_pos + size, current_pos:current_pos + size] = mask
        current_pos += size

    return combined_mask

def block_diagonal_concat_4d(*masks, dtype=torch.bfloat16):
    total_size = sum(mask.size(1) for mask in masks)
    combined_mask = torch.zeros(masks[0].shape[0], 
                                total_size, total_size, dtype=dtype)

    current_pos = 0

    for mask in masks:
        size = mask.size(1)
        combined_mask[:, current_pos:current_pos + size, current_pos:current_pos + size] = mask
        current_pos += size

    return combined_mask

def block_diagonal_concat_cross(*masks, dtype=torch.bfloat16):
    total_rows = sum(mask.size(0) for mask in masks)
    total_cols = sum(mask.size(1) for mask in masks)
    
    combined_mask = torch.zeros((total_rows, total_cols), dtype=dtype)
    
    current_row, current_col = 0, 0

    for mask in masks:
        rows, cols = mask.size()
        combined_mask[current_row:current_row + rows, current_col:current_col + cols] = mask
        current_row += rows
        current_col += cols
        
    return combined_mask

def multipack(input_ids, labels, lengths):
    results = {
        'input_ids': input_ids,
        'labels': labels,
    }
    attention_mask = []
    encoder_attention_mask = []
    decoder_attention_mask = []
    encoder_biases = []
    decoder_biases = []
    
    for length in lengths:
        left_len = length[0]
        right_len = length[1]
        
        attention_mask.append(torch.ones(left_len, left_len))
        encoder_attention_mask.append(torch.ones(right_len, left_len))
        decoder_attention_mask.append(torch.tril(torch.ones(right_len, right_len)))
        
        encoder_bias = compute_bias(
            left_len, left_len,
            encoder_emb,
            bidirectional=True,
            num_buckets=model.config.relative_attention_num_buckets,
            max_distance=model.config.relative_attention_max_distance,
        )
        encoder_biases.append(encoder_bias[0])
        
        decoder_bias = compute_bias(
            right_len, right_len,
            decoder_emb,
            bidirectional=False,
            num_buckets=model.config.relative_attention_num_buckets,
            max_distance=model.config.relative_attention_max_distance,
        )
        decoder_biases.append(decoder_bias[0])
        
    results['attention_mask'] = block_diagonal_concat(*attention_mask)
    results['encoder_attention_mask'] = block_diagonal_concat_cross(*encoder_attention_mask)
    results['decoder_attention_mask'] = block_diagonal_concat(*decoder_attention_mask)
    
    results['position_bias'] = block_diagonal_concat_4d(*encoder_biases)
    results['decoder_position_bias'] = block_diagonal_concat_4d(*decoder_biases)
    
    return results

def collator(batch, pad_token_id = 1, label_pad = -100):
    max_length = max(len(l['input_ids']) for l in batch)
    results = {}
    results['input_ids'] = [
        b['input_ids'] + [pad_token_id] * (max_length - len(b['input_ids']))
        for b in batch
    ]
    results['input_ids'] = torch.tensor(results['input_ids'], dtype = torch.int64)
    
    max_length = max(len(l['labels']) for l in batch)
    results['labels'] = [
        b['labels'] + [label_pad] * (max_length - len(b['labels']))
        for b in batch
    ]
    results['labels'] = torch.tensor(results['labels'], dtype = torch.int64)
    
    results['position_bias'] = pad_attention_mask_4d([b['position_bias'] for b in batch])
    results['decoder_position_bias'] = pad_attention_mask_4d([b['decoder_position_bias'] for b in batch])
    
    attention_mask = [b['attention_mask'] for b in batch]
    results['attention_mask'] = pad_attention_mask(attention_mask)
    encoder_attention_mask = [b['encoder_attention_mask'] for b in batch]
    results['encoder_attention_mask'] = pad_attention_mask(encoder_attention_mask)
    decoder_attention_mask = [b['decoder_attention_mask'] for b in batch]
    results['decoder_attention_mask'] = pad_attention_mask(decoder_attention_mask)
    
    dtype = results['attention_mask'].dtype
    encoder_extended_attention_mask = results['attention_mask'][:, None, :, :]
    encoder_extended_attention_mask = encoder_extended_attention_mask
    encoder_extended_attention_mask = (1.0 - encoder_extended_attention_mask) * torch.finfo(dtype).min
    results['position_bias'] = results['position_bias'] + encoder_extended_attention_mask
    
    dtype = results['decoder_attention_mask'].dtype
    encoder_extended_attention_mask = results['decoder_attention_mask'][:, None, :, :]
    encoder_extended_attention_mask = encoder_extended_attention_mask
    encoder_extended_attention_mask = (1.0 - encoder_extended_attention_mask) * torch.finfo(dtype).min
    results['decoder_position_bias'] = results['decoder_position_bias'] + encoder_extended_attention_mask
    
    return results

In [27]:
!mkdir tokenized-4096

In [28]:
def loop(files, block_size = 4096):
    files, index = files
    
    out_root = f'tokenized-4096/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    
    count_input_ids = 0
    count_labels = 0
    input_ids = []
    labels = []
    lengths = []
    found = False
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for f in files:
            with open(f) as fopen:
                for l in tqdm(fopen):
                    try:
                        l = json.loads(l)['translation']
                        left = l['prefix'] + l['src'] + tokenizer.eos_token
                        right = l['tgt'] + tokenizer.eos_token

                        left = tokenizer(left)['input_ids']
                        right = tokenizer(right)['input_ids']
                        
                        if len(left) > block_size or len(right) > block_size:
                            continue
                    except:
                        continue

                    if count_input_ids + len(left) >= block_size or count_labels + len(left) >= block_size:
                        
                        if len(input_ids) and len(labels):
                            d = {
                                'input_ids': input_ids,
                                'labels': labels,
                                'lengths': lengths
                            }
                            count_input_ids = len(left)
                            count_labels = len(right)
                            input_ids = left
                            labels = right
                            lengths = [(len(left), len(right))]

                            d = json.dumps(d)

                            out.write({
                                'data': d,
                            })       

                    else:
                        count_input_ids += len(left)
                        count_labels += len(right)
                        input_ids.extend(left)
                        labels.extend(right)
                        lengths.append((len(left), len(right)))

In [29]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [30]:
import mp

In [ ]:
mp.multiprocessing(files, loop, cores = 15, returned = False)

1000000it [03:49, 4366.66it/s]
1000000it [04:02, 4131.46it/s]
1000000it [04:04, 4085.34it/s]
1000000it [04:35, 3632.01it/s]
1000000it [05:16, 3157.30it/s]
1000000it [05:18, 3136.65it/s]
1000000it [05:38, 2955.02it/s]
1000000it [06:00, 2774.71it/s]
1000000it [06:14, 2669.13it/s]
1000000it [06:15, 2660.57it/s]
1000000it [06:45, 2467.39it/s]
1000000it [07:11, 2316.30it/s]
1000000it [07:16, 2289.88it/s]
1000000it [04:02, 4118.25it/s]
1000000it [04:24, 3784.81it/s]
1000000it [04:31, 3688.85it/s]
1000000it [08:57, 1861.66it/s]
1000000it [02:42, 6164.25it/s]
1000000it [04:25, 3772.05it/s]
1000000it [03:17, 5062.00it/s]
1000000it [10:12, 1631.59it/s]
1000000it [05:10, 3216.73it/s]
1000000it [03:24, 4887.27it/s]
1000000it [05:27, 3055.80it/s]
1000000it [04:17, 3879.38it/s]
1000000it [05:26, 3065.65it/s]
1000000it [03:35, 4643.63it/s]
1000000it [02:53, 5773.68it/s]
1000000it [02:52, 5803.79it/s]
1000000it [03:53, 4274.86it/s]
1000000it [03:37, 4604.60it/s]
1000000it [02:41, 6200.34it/s]
1000000i

In [18]:
folders = sorted(glob('tokenized/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

['tokenized/tokenized-0',
 'tokenized/tokenized-1',
 'tokenized/tokenized-2',
 'tokenized/tokenized-3',
 'tokenized/tokenized-4',
 'tokenized/tokenized-5',
 'tokenized/tokenized-6',
 'tokenized/tokenized-7',
 'tokenized/tokenized-8',
 'tokenized/tokenized-9',
 'tokenized/tokenized-10',
 'tokenized/tokenized-11',
 'tokenized/tokenized-12',
 'tokenized/tokenized-13',
 'tokenized/tokenized-14',
 'tokenized/tokenized-15',
 'tokenized/tokenized-16']

In [19]:
!rm -rf packing

In [20]:
with MDSWriter(out='packing', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|███████████████████████████████████████████████████████████████████████████████████| 191841/191841 [00:21<00:00, 8797.53it/s]


In [21]:
dataset = LocalDataset('/home/husein/mesolitica/t5-sdpa-multipack/packing')

In [22]:
len(dataset)

2390857

In [23]:
d = json.loads(dataset[0]['data'])
b = multipack(**d)
d = json.loads(dataset[1]['data'])
b1 = multipack(**d)

In [24]:
%%time

input_ids = collator([b])

CPU times: user 1.44 s, sys: 95.5 ms, total: 1.53 s
Wall time: 163 ms


In [26]:
input_ids['input_ids'].shape

torch.Size([1, 2001])

In [2]:
from huggingface_hub import create_repo, delete_repo

try:
    delete_repo(repo_id="mesolitica/malaysian-translation-v2-multipack-2048", repo_type="dataset")
except:
    pass
create_repo("mesolitica/malaysian-translation-v2-multipack-2048", repo_type="dataset", private = True)

RepoUrl('https://huggingface.co/datasets/mesolitica/malaysian-translation-v2-multipack-2048', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/malaysian-translation-v2-multipack-2048')

In [6]:
!du -hs packing

43G	packing


In [7]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="packing",
    repo_id="mesolitica/malaysian-translation-v2-multipack-2048",
    repo_type="dataset",
)

shard.00003.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00001.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00002.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 682 LFS files:   0%|          | 0/682 [00:00<?, ?it/s]

shard.00004.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

shard.00678.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00679.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00680.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00681.mds:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/malaysian-translation-v2-multipack-2048/commit/a05a0803969aa5b4720b825d1f5cd7a15e0e11a4', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a05a0803969aa5b4720b825d1f5cd7a15e0e11a4', pr_url=None, pr_revision=None, pr_num=None)